In [1]:
import scanpy as sc
import anndata
import numpy as np
import gc
import pandas as pd 
import os
from datetime import date
from matplotlib.pyplot import rc_context
import torch
import scarches as sca
import gdown
gc.isenabled()

 captum (see https://github.com/pytorch/captum).
INFO:lightning_fabric.utilities.seed:Global seed set to 0


True

In [13]:
pd.set_option('display.max_columns', None)

In [ ]:
path_project = '/nfs/research/irene/anaelle/CrossSpeciesIC'
print(path_project)
path_scripts = os.path.join(path_project, 'expiMap','object_human_mouse')
print(path_scripts)
path_data = os.path.join(path_project, 'data')
print(path_data)

In [3]:
adata = sc.read_h5ad(os.path.join(path_data,'all_homolog_human_mouse.h5ad'))

In [16]:
adata

AnnData object with n_obs × n_vars = 236271 × 16855
    obs: 'sample_name', 'exp_component_name', 'cluster_label', 'cluster_color', 'cluster_order', 'class_label', 'class_color', 'class_order', 'subclass_label', 'subclass_color', 'subclass_order', 'donor_sex_label', 'donor_sex_color', 'donor_sex_order', 'region_label', 'region_color', 'region_order', 'cortical_layer_label', 'cortical_layer_color', 'cortical_layer_order', 'cell_type_accession_label', 'cell_type_accession_color', 'cell_type_accession_order', 'cell_type_alias_label', 'cell_type_alias_color', 'cell_type_alias_order', 'cell_type_alt_alias_label', 'cell_type_alt_alias_color', 'cell_type_alt_alias_order', 'cell_type_designation_label', 'cell_type_designation_color', 'cell_type_designation_order', 'external_donor_name_label', 'external_donor_name_color', 'external_donor_name_order', 'specimen_type', 'full_genotype_label', 'outlier_type', 'homolog_class_label', 'homolog_subclass_label', 'Unnamed: 0', 'nUMI', 'nGene', 'QC', 'clu

In [4]:
adata.obs[['homolog_subclass_label']].value_counts()

homolog_subclass_label
L2/3 IT                   59856
L5 IT                     32777
Oligo                     23319
L6 CT                     20473
Astro                     18369
Pvalb                     13555
Sst                       10112
Macrophage                 8128
Vip                        7690
L6 IT                      6916
Lamp5                      6910
L5 ET                      5374
OPC                        5027
L5/6 NP                    4772
L6b                        3748
Endo                       3183
VLMC                       2383
Sncg                       1450
Peri                       1450
L6 IT Car3                  327
Prog/IP                     246
Micro-PVM                   108
Sst Chodl                    67
CR                           16
SMC                          15
Name: count, dtype: int64

In [8]:
adatas = [adata[adata.obs['homolog_subclass_label']== clust] for clust in adata.obs['homolog_subclass_label'].cat.categories]
for dat in adatas:
    if dat.n_obs > 350:
         sc.pp.subsample(dat, fraction=0.2)
adata_downsampled = anndata.concat([adatas[0],*adatas[1:]])

In [9]:
adata_downsampled

AnnData object with n_obs × n_vars = 47870 × 16855
    obs: 'sample_name', 'exp_component_name', 'cluster_label', 'cluster_color', 'cluster_order', 'class_label', 'class_color', 'class_order', 'subclass_label', 'subclass_color', 'subclass_order', 'donor_sex_label', 'donor_sex_color', 'donor_sex_order', 'region_label', 'region_color', 'region_order', 'cortical_layer_label', 'cortical_layer_color', 'cortical_layer_order', 'cell_type_accession_label', 'cell_type_accession_color', 'cell_type_accession_order', 'cell_type_alias_label', 'cell_type_alias_color', 'cell_type_alias_order', 'cell_type_alt_alias_label', 'cell_type_alt_alias_color', 'cell_type_alt_alias_order', 'cell_type_designation_label', 'cell_type_designation_color', 'cell_type_designation_order', 'external_donor_name_label', 'external_donor_name_color', 'external_donor_name_order', 'specimen_type', 'full_genotype_label', 'outlier_type', 'homolog_class_label', 'homolog_subclass_label', 'Unnamed: 0', 'nUMI', 'nGene', 'QC', 'clus

In [10]:
adata_downsampled.var_names

Index(['ENSG00000121410', 'ENSG00000148584', 'ENSG00000175899',
       'ENSG00000184389', 'ENSG00000128274', 'ENSG00000118017',
       'ENSG00000094914', 'ENSG00000081760', 'ENSG00000114771',
       'ENSG00000197953',
       ...
       'ENSG00000083812', 'ENSG00000181273', 'ENSG00000142789',
       'ENSG00000163606', 'ENSG00000170465', 'ENSG00000143149',
       'ENSG00000274736', 'ENSG00000212124', 'ENSG00000072694',
       'ENSG00000178586'],
      dtype='object', length=16855)

In [11]:
adata_downsampled.obs_names

Index(['pBICCNsMMrMOpRMiM004d190318_TCTGGCTGTCCGCAGT',
       'pBICCNsMMrMOpRMiM008d190320_GTTCGCTTCCGAGCTG',
       'pBICCNsMMrMOpRAiF007d190314_AGGGCCTCATGCCGAC',
       'pBICCNsMMrMOpRAiM007d190320_GAATAGATCTAGTCAG',
       'pBICCNsMMrMOpRAiM007d190320_TGCGATATCTCTGAGA',
       'pBICCNsMMrMOpRPiF008d190314_CACAACACATCGAGCC',
       'pBICCNsMMrMOpRMiF006d190314_GTTCGCTCATAGAAAC',
       'pBICCNsMMrMOpRMiF003d190318_TTACCGCGTCGATTTG',
       'pBICCNsMMrMOpRPiF005d190318_TGGTACAGTAGTGATA',
       'pBICCNsMMrMOpRPiF007d190314_GAAATGATCATGACAC',
       ...
       'pBICCNsMMrMOpRMiF006d190314_CCCATTGCAGCTTCCT', '76348', '31528',
       '51165', 'pBICCNsMMrMOpRAiF007d190314_ACCAACATCGACTCCT',
       'pBICCNsMMrMOpRAiM003d190318_ATTCCATGTTGTTGCA', '38369',
       'pBICCNsMMrMOpRAiM003d190318_TTACGTTTCAGTGTTG', '30940', '522'],
      dtype='object', length=47870)

In [14]:
adata_downsampled.obs

,sample_name,exp_component_name,cluster_label,cluster_color,cluster_order,class_label,class_color,class_order,subclass_label,subclass_color,subclass_order,donor_sex_label,donor_sex_color,donor_sex_order,region_label,region_color,region_order,cortical_layer_label,cortical_layer_color,cortical_layer_order,cell_type_accession_label,cell_type_accession_color,cell_type_accession_order,cell_type_alias_label,cell_type_alias_color,cell_type_alias_order,cell_type_alt_alias_label,cell_type_alt_alias_color,cell_type_alt_alias_order,cell_type_designation_label,cell_type_designation_color,cell_type_designation_order,external_donor_name_label,external_donor_name_color,external_donor_name_order,specimen_type,full_genotype_label,outlier_type,homolog_class_label,homolog_subclass_label,Unnamed: 0,nUMI,nGene,QC,cluster,Allen.cluster_id,Allen.cluster_label,Allen.class_label,Allen.subclass_label,comb.QC,row,BICCN_cluster_id,BICCN_cluster_label,BICCN_class_label,BICCN_subclass_label,size,gene.counts,umi.counts,Broad.QC.doublet,Broad.QC.Mito,Broad.passQC,MALE,Comb.QC,cl,temp_class_label,BICCN_ontology_term_id,assay_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,self_reported_ethnicity_ontology_term_id,development_stage_ontology_term_id,sex_ontology_term_id,organism_ontology_term_id,donor_id,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,species
pBICCNsMMrMOpRMiM004d190318_TCTGGCTGTCCGCAGT,pBICCNsMMrMOpRMiM004d190318_TCTGGCTGTCCGCAGT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Neuronal,Astro,35925.0,6610.0,2764.0,PassQC,Astro_1,91.0,Astro Aqp4_1,Non-Neuronal,Astro,PassQC,132234.0,91.0,Astro Aqp4_1,Non-Neuronal,Astro,16143.0,2601.086105,5846.112804,0.03,0.0,0.97,0.448615,NaN,113.0,Non-NeuronalAstro,ILX:0770141,EFO:0009922,PATO:0000461,UBERON:0001384,CL:0000127,na,MmusDv:0000061,PATO:0000384,NCBITaxon:10090,M004,nucleus,astrocyte,10x 3' v3,normal,Mus musculus,male,primary motor cortex,na,early adult stage,mouse
pBICCNsMMrMOpRMiM008d190320_GTTCGCTTCCGAGCTG,pBICCNsMMrMOpRMiM008d190320_GTTCGCTTCCGAGCTG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Neuronal,Astro,79695.0,4208.0,2208.0,PassQC,Astro_0,91.0,Astro Aqp4_1,Non-Neuronal,Astro,PassQC,176004.0,91.0,Astro Aqp4_1,Non-Neuronal,Astro,16143.0,2601.086105,5846.112804,0.03,0.0,0.97,0.448615,NaN,113.0,Non-NeuronalAstro,ILX:0770141,EFO:0009922,PATO:0000461,UBERON:0001384,CL:0000127,na,MmusDv:0000061,PATO:0000384,NCBITaxon:10090,M008,nucleus,astrocyte,10x 3' v3,normal,Mus musculus,male,primary motor cortex,na,early adult stage,mouse
pBICCNsMMrMOpRAiF007d190314_AGGGCCTCATGCCGAC,pBICCNsMMrMOpRAiF007d190314_AGGGCCTCATGCCGAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Neuronal,Astro,103327.0,6280.0,2749.0,PassQC,Astro_4,91.0,Astro Aqp4_1,Non-Neuronal,Astro,PassQC,2858.0,91.0,Astro Aqp4_1,Non-Neuronal,Astro,16143.0,2601.086105,5846.112804,0.03,0.0,0.97,0.448615,NaN,113.0,Non-NeuronalAstro,ILX:0770141,EFO:0009922,PATO:0000461,UBERON:0001384,CL:0000127,na,MmusDv:0000061,PATO:0000383,NCBITaxon:10090,F007,nucleus,astrocyte,10x 3' v3,normal,Mus musculus,female,primary motor cortex,na,early adult stage,mouse
pBICCNsMMrMOpRAiM007d190320_GAATAGATCTAGTCAG,pBICCNsMMrMOpRAiM007d190320_GAATAGATCTAGTCAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Neuronal,Astro,50219.0,9993.0,3773.0,PassQC,Astro_4,91.0,Astro Aqp4_1,Non-Neuronal,Astro,PassQC,146528.0,91.0,Astro Aqp4_1,Non-Neuronal,Astro,16143.0,2601.086105,5846.112804,0.03,0.0,0.97,0.448615,NaN,113.0,Non-NeuronalAstro,ILX:0770141,EFO:0009922,PATO:0000461,U

In [15]:
adata_downsampled.obs[['homolog_subclass_label']].value_counts()

homolog_subclass_label
L2/3 IT                   11971
L5 IT                      6555
Oligo                      4663
L6 CT                      4094
Astro                      3673
Pvalb                      2711
Sst                        2022
Macrophage                 1625
Vip                        1538
L6 IT                      1383
Lamp5                      1382
L5 ET                      1074
OPC                        1005
L5/6 NP                     954
L6b                         749
Endo                        636
VLMC                        476
L6 IT Car3                  327
Sncg                        290
Peri                        290
Prog/IP                     246
Micro-PVM                   108
Sst Chodl                    67
CR                           16
SMC                          15
Name: count, dtype: int64

In [17]:
adata_downsampled.var

""
ENSG00000121410
ENSG00000148584
ENSG00000175899
ENSG00000184389
ENSG00000128274
...
ENSG00000143149
ENSG00000274736
ENSG00000212124
ENSG00000072694


In [18]:
adata.var

,human_chromosome,human_long_gene_name,human_gene_name,mouse_ensembl_id,orthology_type,mouse_gene_name,human_entrez_id,human_ensembl_id,mouse_homologs_ids,mouse_homologs_names,human_homologs_ensembl_ids,human_homologs_entrez_ids,human_homologs_names,batch
ENSG00000121410,19,alpha-1-B glycoprotein,A1BG,ENSMUSG00000022347,ortholog_one2one,A1bg,1.0,ENSG00000121410,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000148584,10,APOBEC1 complementation factor,A1CF,ENSMUSG00000052595,ortholog_one2one,A1cf,29974.0,ENSG00000148584,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000175899,12,alpha-2-macroglobulin,A2M,ENSMUSG00000030111,ortholog_one2one,A2m,2.0,ENSG00000175899,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000184389,1,"alpha 1,3-galactosyltransferase 2",A3GALT2,ENSMUSG00000028794,ortholog_one2one,A3galt2,127550.0,ENSG00000184389,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000128274,22,"alpha 1,4-galactosyltransferase",A4GALT,ENSMUSG00000047878,ortholog_one2one,A4galt,53947.0,ENSG00000128274,NaN,NaN,NaN,NaN,NaN,o2o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000143149,1,"aldehyde dehydrogenase 9 family, member A1",ALDH9A1,ENSMUSG00000026687,ortholog_many2many,Aldh9a1,223.0,ENSG00000143149,['ENSMUSG00000026687'],['Aldh9a1'],['ENSG00000143149'],[223.0],['ALDH9A1'],m2m
ENSG00000274736,17,chemokine (C-C motif) ligand 23,CCL23,ENSMUSG00000018927,ortholog_many2many,Ccl6,6368.0,ENSG00000274736,"['ENSMUSG00000018927', 'ENSMUSG00000019122']","['Ccl9', 'Ccl6']","['ENSG00000274736', 'ENSG00000275718']","[6359.0, 6368.0]","['CCL15', 'CCL23']",m2m
ENSG00000212124,12,"taste receptor, type 2, member 19",TAS2R19,ENSMUSG00000053217,ortholog_many2many,Tas2r136,259294.0,ENSG00000212124,"['ENSMUSG00000053217', 'ENSMUSG00000059382']","['Tas2r120', 'Tas2r136']","['ENSG00000212124', 'ENSG00000212126', 'ENSG00...","[259296.0, 259295.0, 259294.0, 259290.0, 25929...","['TAS2R50', 'TAS2R20', 'TAS2R19', 'TAS2R31', '...",m2m
ENSG00000072694,1,"Fc fragment of IgG, low affinity IIb, receptor...",FCGR2B,ENSMUSG00000026656,ortholog_many2many,Fcgr2b,2213.0,ENSG00000072694,"['ENSMUSG00000026656', 'ENSMUSG00000059498']","['Fcgr2b', 'Fcgr3']","['ENSG00000072694', 'ENSG00000143226']","[2213.0, 2212.0]","['FCGR2B', 'FCGR2A']",m2m


In [19]:
sum(adata.var.index == adata_downsampled.var.index)

16855

In [20]:
vardf = adata.var.copy()

In [21]:
vardf

,human_chromosome,human_long_gene_name,human_gene_name,mouse_ensembl_id,orthology_type,mouse_gene_name,human_entrez_id,human_ensembl_id,mouse_homologs_ids,mouse_homologs_names,human_homologs_ensembl_ids,human_homologs_entrez_ids,human_homologs_names,batch
ENSG00000121410,19,alpha-1-B glycoprotein,A1BG,ENSMUSG00000022347,ortholog_one2one,A1bg,1.0,ENSG00000121410,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000148584,10,APOBEC1 complementation factor,A1CF,ENSMUSG00000052595,ortholog_one2one,A1cf,29974.0,ENSG00000148584,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000175899,12,alpha-2-macroglobulin,A2M,ENSMUSG00000030111,ortholog_one2one,A2m,2.0,ENSG00000175899,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000184389,1,"alpha 1,3-galactosyltransferase 2",A3GALT2,ENSMUSG00000028794,ortholog_one2one,A3galt2,127550.0,ENSG00000184389,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000128274,22,"alpha 1,4-galactosyltransferase",A4GALT,ENSMUSG00000047878,ortholog_one2one,A4galt,53947.0,ENSG00000128274,NaN,NaN,NaN,NaN,NaN,o2o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000143149,1,"aldehyde dehydrogenase 9 family, member A1",ALDH9A1,ENSMUSG00000026687,ortholog_many2many,Aldh9a1,223.0,ENSG00000143149,['ENSMUSG00000026687'],['Aldh9a1'],['ENSG00000143149'],[223.0],['ALDH9A1'],m2m
ENSG00000274736,17,chemokine (C-C motif) ligand 23,CCL23,ENSMUSG00000018927,ortholog_many2many,Ccl6,6368.0,ENSG00000274736,"['ENSMUSG00000018927', 'ENSMUSG00000019122']","['Ccl9', 'Ccl6']","['ENSG00000274736', 'ENSG00000275718']","[6359.0, 6368.0]","['CCL15', 'CCL23']",m2m
ENSG00000212124,12,"taste receptor, type 2, member 19",TAS2R19,ENSMUSG00000053217,ortholog_many2many,Tas2r136,259294.0,ENSG00000212124,"['ENSMUSG00000053217', 'ENSMUSG00000059382']","['Tas2r120', 'Tas2r136']","['ENSG00000212124', 'ENSG00000212126', 'ENSG00...","[259296.0, 259295.0, 259294.0, 259290.0, 25929...","['TAS2R50', 'TAS2R20', 'TAS2R19', 'TAS2R31', '...",m2m
ENSG00000072694,1,"Fc fragment of IgG, low affinity IIb, receptor...",FCGR2B,ENSMUSG00000026656,ortholog_many2many,Fcgr2b,2213.0,ENSG00000072694,"['ENSMUSG00000026656', 'ENSMUSG00000059498']","['Fcgr2b', 'Fcgr3']","['ENSG00000072694', 'ENSG00000143226']","[2213.0, 2212.0]","['FCGR2B', 'FCGR2A']",m2m


In [23]:
sum(vardf.reindex(adata_downsampled.var.index).index == adata_downsampled.var.index)

16855

In [24]:
adata_downsampled.var = vardf.reindex(adata_downsampled.var.index)

In [25]:
adata_downsampled.var

,human_chromosome,human_long_gene_name,human_gene_name,mouse_ensembl_id,orthology_type,mouse_gene_name,human_entrez_id,human_ensembl_id,mouse_homologs_ids,mouse_homologs_names,human_homologs_ensembl_ids,human_homologs_entrez_ids,human_homologs_names,batch
ENSG00000121410,19,alpha-1-B glycoprotein,A1BG,ENSMUSG00000022347,ortholog_one2one,A1bg,1.0,ENSG00000121410,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000148584,10,APOBEC1 complementation factor,A1CF,ENSMUSG00000052595,ortholog_one2one,A1cf,29974.0,ENSG00000148584,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000175899,12,alpha-2-macroglobulin,A2M,ENSMUSG00000030111,ortholog_one2one,A2m,2.0,ENSG00000175899,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000184389,1,"alpha 1,3-galactosyltransferase 2",A3GALT2,ENSMUSG00000028794,ortholog_one2one,A3galt2,127550.0,ENSG00000184389,NaN,NaN,NaN,NaN,NaN,o2o
ENSG00000128274,22,"alpha 1,4-galactosyltransferase",A4GALT,ENSMUSG00000047878,ortholog_one2one,A4galt,53947.0,ENSG00000128274,NaN,NaN,NaN,NaN,NaN,o2o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000143149,1,"aldehyde dehydrogenase 9 family, member A1",ALDH9A1,ENSMUSG00000026687,ortholog_many2many,Aldh9a1,223.0,ENSG00000143149,['ENSMUSG00000026687'],['Aldh9a1'],['ENSG00000143149'],[223.0],['ALDH9A1'],m2m
ENSG00000274736,17,chemokine (C-C motif) ligand 23,CCL23,ENSMUSG00000018927,ortholog_many2many,Ccl6,6368.0,ENSG00000274736,"['ENSMUSG00000018927', 'ENSMUSG00000019122']","['Ccl9', 'Ccl6']","['ENSG00000274736', 'ENSG00000275718']","[6359.0, 6368.0]","['CCL15', 'CCL23']",m2m
ENSG00000212124,12,"taste receptor, type 2, member 19",TAS2R19,ENSMUSG00000053217,ortholog_many2many,Tas2r136,259294.0,ENSG00000212124,"['ENSMUSG00000053217', 'ENSMUSG00000059382']","['Tas2r120', 'Tas2r136']","['ENSG00000212124', 'ENSG00000212126', 'ENSG00...","[259296.0, 259295.0, 259294.0, 259290.0, 25929...","['TAS2R50', 'TAS2R20', 'TAS2R19', 'TAS2R31', '...",m2m
ENSG00000072694,1,"Fc fragment of IgG, low affinity IIb, receptor...",FCGR2B,ENSMUSG00000026656,ortholog_many2many,Fcgr2b,2213.0,ENSG00000072694,"['ENSMUSG00000026656', 'ENSMUSG00000059498']","['Fcgr2b', 'Fcgr3']","['ENSG00000072694', 'ENSG00000143226']","[2213.0, 2212.0]","['FCGR2B', 'FCGR2A']",m2m


In [26]:
adata_downsampled

AnnData object with n_obs × n_vars = 47870 × 16855
    obs: 'sample_name', 'exp_component_name', 'cluster_label', 'cluster_color', 'cluster_order', 'class_label', 'class_color', 'class_order', 'subclass_label', 'subclass_color', 'subclass_order', 'donor_sex_label', 'donor_sex_color', 'donor_sex_order', 'region_label', 'region_color', 'region_order', 'cortical_layer_label', 'cortical_layer_color', 'cortical_layer_order', 'cell_type_accession_label', 'cell_type_accession_color', 'cell_type_accession_order', 'cell_type_alias_label', 'cell_type_alias_color', 'cell_type_alias_order', 'cell_type_alt_alias_label', 'cell_type_alt_alias_color', 'cell_type_alt_alias_order', 'cell_type_designation_label', 'cell_type_designation_color', 'cell_type_designation_order', 'external_donor_name_label', 'external_donor_name_color', 'external_donor_name_order', 'specimen_type', 'full_genotype_label', 'outlier_type', 'homolog_class_label', 'homolog_subclass_label', 'Unnamed: 0', 'nUMI', 'nGene', 'QC', 'clus

In [27]:
adata_downsampled.write_h5ad(os.path.join(path_data,'all_hhm_reduced.h5ad'),compression='gzip')